# **Finetuning VITS Text-to-Speech Model in Vietnamese**
@phatjk - Nguyễn Thành Phát
## (Nhấn CRTL + F9 để chạy tất cả)

# **Install packages**

In [1]:
# Setup Evironments
!git clone -q https://github.com/rmcpantoja/piper
%cd /content/piper/src/python
!wget -q "https://raw.githubusercontent.com/coqui-ai/TTS/dev/TTS/bin/resample.py"
!pip install -q cython>=0.29.0 piper-phonemize==1.1.0 librosa>=0.9.2 numpy>=1.19.0 onnxruntime==1.15.1 pytorch-lightning==1.7.0 torch==1.11.0
!pip install -q torchtext==0.12.0 torchvision==0.12.0
!pip install -q torchaudio==0.11.0 torchmetrics==0.11.4
!pip install -q textgrid
!bash build_monotonic_align.sh
!apt-get install -q espeak-ng

/content/piper/src/python
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 21.0 MB/s eta 0:00:00
Compiling /content/piper/src/python/piper_train/vits/monotonic_align/core.pyx because it changed.
[1/1] Cyth

# **Download datasets**

In [2]:
!gdown 1Pe-5lKT_lZsliv2WxQDai2mjhI9ZMFlj
!unzip reinfo.zip -d wav/

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: wav/09498.wav           
  inflating: wav/08789.wav           
  inflating: wav/05317.TextGrid      
  inflating: wav/13339.TextGrid      
  inflating: wav/01585.wav           
  inflating: wav/08844.TextGrid      
  inflating: wav/05242.TextGrid      
  inflating: wav/03398.TextGrid      
  inflating: wav/02933.TextGrid      
  inflating: wav/08821.TextGrid      
  inflating: wav/13392.wav           
  inflating: wav/00707.TextGrid      
  inflating: wav/12514.wav           
  inflating: wav/06335.TextGrid      
  inflating: wav/04744.TextGrid      
  inflating: wav/05098.wav           
  inflating: wav/03930.wav           
  inflating: wav/03492.TextGrid      
  inflating: wav/14473.wav           
  inflating: wav/12696.wav           
  inflating: wav/00161.wav           
  inflating: wav/08032.TextGrid      
  inflating: wav/07103.wav           
  inflating: wav/06629.wav           
  inflating: wav/05475.TextGrid

# **Data preprocessing**

In [3]:
import os
dir_path = r'wav'
res = []
# Lấy tất cả tên file
for file_path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, file_path)):
        res.append(file_path)

list_text_files = list(filter(lambda x: "TextGrid" in x,res))

# Chỉ cắt 500 điểm dữ liệu để sử dụng finetuining
list_text_files = list_text_files[:500]

# Lấy text của audio từ file TextGrid
import textgrid
all_id = []
all_text = []
for text_file in list_text_files:
  tg = textgrid.TextGrid.fromFile('wav/'+text_file)
  text = ""
  for i in range(len(tg[0])):
    if tg[0][i].mark != '':
      text += tg[0][i].mark + " "
  file_name = text_file.replace(".TextGrid","")
  all_id.append(file_name)
  all_text.append(text)

In [4]:
# Chuyển dữ liệu thành dạng dataframe để dễ xuất file
import pandas as pd

data = {
  "id": all_id,
  "text": all_text
}
df = pd.DataFrame(data)

print(df)

        id                                               text
0    13178  bày tỏ không thích nhưng các giám khảo lại đán...
1    00113  không nhảm lấy gì để cười hài mà phân biệt hài...
2    11626  dù được cán bộ đưa vào bệnh viện cấp cứu ngay ...
3    08788  để làm đẹp nội thất hiện nay đang được nhiều n...
4    11072  yếu tố an toàn của nam châm nâng tay gạt đạt g...
..     ...                                                ...
495  08065  nếu bạn nhìn kỹ văn bản trong thiết kế có độ d...
496  06338  những con cừu sống trong tử cung nhân tạo có t...
497  06369  lương thưởng cạnh tranh cơ hội thăng tiến cao ...
498  14396  trường học dạy võ thoát khỏi yêu râu xanh cho ...
499  13566  rửa mũi với nước muối ăn giúp rửa sạch hốc mũi...

[500 rows x 2 columns]


In [5]:
# Xuất file data
df.to_csv('metadata.csv', sep='|', index=False,header=False)

In [6]:
!pwd

/content/piper/src/python


# **Resampling datasets**

In [7]:
# Đưa sampling rate của audio về 22050
resample = True
sample_rate = 22050
if resample:
  !python resample.py --input_dir "wav" --output_dir "wavs_resampled" --output_sr {sample_rate} --file_ext "wav"
  !mv wavs_resampled wav

Recursively copying the input folder...
Resampling the audio files...
Found 14935 files...
100% 14935/14935 [02:39<00:00, 93.63it/s] 
Done !


# **Preprocess text to Phonemes**

In [8]:
# Chuyển dữ liệu text sang Phonemes (âm vị)
dataset_format = "ljspeech"
force_sp = "--single-speaker"
!python -m piper_train.preprocess \
  --language vi \
  --input-dir /content/piper/src/python \
  --output-dir /content/piper/src/python \
  --dataset-format {dataset_format} \
  --sample-rate {sample_rate} \
  {force_sp}

INFO:preprocess:Single speaker dataset
INFO:preprocess:Wrote dataset config
INFO:preprocess:Processing 500 utterance(s) with 2 worker(s)


# **Download pretrained model**

In [9]:
# Tải pretrain model để tiếng hành finetuning
!wget https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/vi/vi_VN/vais1000/medium/epoch%3D4769-step%3D919580.ckpt
!wget https://huggingface.co/datasets/rhasspy/piper-checkpoints/raw/main/vi/vi_VN/vais1000/medium/config.json

--2023-09-27 06:47:59--  https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/vi/vi_VN/vais1000/medium/epoch%3D4769-step%3D919580.ckpt
Resolving huggingface.co (huggingface.co)... 18.164.174.23, 18.164.174.55, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/6c/3b/6c3bb73012be460f91a23801a60eb3831f76d24f6812c18e0ca05b16673ae99f/c3879268594a85b5133146e6a4709a5cc2f3014ef0eabf9280f67289054ba2bc?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27epoch%253D4769-step%253D919580.ckpt%3B+filename%3D%22epoch%3D4769-step%3D919580.ckpt%22%3B&Expires=1696056479&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NjA1NjQ3OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy82Yy8zYi82YzNiYjczMDEyYmU0NjBmOTFhMjM4MDFhNjBlYjM4MzFmNzZkMjRmNjgxMmMxOGUwY2EwNWIxNjY3M2FlOTlmL2

In [10]:
# Đổi tên pretrained model
!mv epoch=4769-step=919580.ckpt pretrained_vi.ckpt

# **Training model**

In [ ]:
#x-low - 16Khz audio, 5-7M params
#medium - 22.05Khz audio, 15-20 params
#high - 22.05Khz audio, 28-32M params
# checkpoint-epochs: Save checkpoint every N epochs (default: 1)

Cài đặt thông số và tiếng hành training

In [11]:
!python -m piper_train \
--dataset-dir "/content/piper/src/python/" \
--accelerator 'gpu' \
--devices 1 \
--batch-size 34 \
--validation-split 0.2 \
--num-test-examples 2 \
--quality "medium" \
--checkpoint-epochs 5 \
--log_every_n_steps 50 \
--max_epochs 5000 \
--resume_from_checkpoint "/content/piper/src/python/pretrained_vi.ckpt"\
--precision 32

DEBUG:piper_train:Namespace(dataset_dir='/content/piper/src/python/', checkpoint_epochs=5, quality='medium', resume_from_single_speaker_checkpoint=None, logger=True, enable_checkpointing=True, default_root_dir=None, gradient_clip_val=None, gradient_clip_algorithm=None, num_nodes=1, num_processes=None, devices='1', gpus=None, auto_select_gpus=False, tpu_cores=None, ipus=None, enable_progress_bar=True, overfit_batches=0.0, track_grad_norm=-1, check_val_every_n_epoch=1, fast_dev_run=False, accumulate_grad_batches=None, max_epochs=5000, min_epochs=None, max_steps=-1, min_steps=None, max_time=None, limit_train_batches=None, limit_val_batches=None, limit_test_batches=None, limit_predict_batches=None, val_check_interval=None, log_every_n_steps=50, accelerator='gpu', strategy=None, sync_batchnorm=False, precision=32, enable_model_summary=True, weights_save_path=None, num_sanity_val_steps=2, resume_from_checkpoint='/content/piper/src/python/pretrained_vi.ckpt', profiler=None, benchmark=None, de

# **View training information on Tensorboard**

In [12]:
%load_ext tensorboard
%tensorboard --logdir /content/piper/src/python/lightning_logs --port=8008

<IPython.core.display.Javascript object>

# **Export ONNX model**

In [13]:
# Export onnx trên model finetuning (Nhớ đổi tên file cho đúng checkpoint)
!python3 -m piper_train.export_onnx \
    /content/piper/src/python/lightning_logs/version_0/checkpoints/epoch=4869-step=921980.ckpt \
    /content/finetuning_pretrained_vi.onnx

!cp /content/piper/src/python/config.json \
   /content/finetuning_pretrained_vi.onnx.json

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmp6mgx9ntb
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmp6mgx9ntb/_remote_module_non_sriptable.py
Removing weight norm...
/content/piper/src/python/piper_train/vits/attentions.py:235: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  t_s == t_t
/content/piper/src/python/piper_train/vits/attentions.py:295: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  pad_length = max(length - (self.window_size + 1), 0)
/content/piper/src/python/piper_train/vits/attentions.py:296:

In [ ]:
# Export onnx trên model gốc
# !python3 -m piper_train.export_onnx \
#     /content/piper/src/python/pretrained_vi.ckpt \
#     /content/finetuning_pretrained_vi.onnx

# !cp /content/piper/src/python/config.json \
#    /content/finetuning_pretrained_vi.onnx.json

# **Test inference**

In [14]:
!pip install piper-tts

In [15]:
!echo 'Xin chào! Đây là âm thanh kiểm tra chất lượng của mô hình tổng hợp tiếng nói trên văn bản tiếng Việt được trích xuất sang định dạng ONNX' | \
  piper -m /content/finetuning_pretrained_vi.onnx --output_file /content/test.wav

In [16]:
from IPython.display import Audio
Audio("/content/test.wav")